# Package Load

In [1]:
import preprocessing
import tensorflow as tf
import numpy as np
import random
import sys
import utils
import pickle

# preprocessing : 이미지 파일로 존재하는 데이터를 numpy와 onehotencoding 라벨로 바꿔주는 모듈
# tensorflow : 텐서플로우 , 일반적으로 tf로 줄여서 사용
# numpy : fundamental package for scientific computing with Python
# pickle : python 의 데이터 형식을 그대로 저장하기 위한 파이썬 기본 패키지

# Image Loading

In [2]:
train_imgs , train_labs , test_imgs , test_labs=preprocessing.get_cifar_data()

train_imgs=np.asarray(train_imgs).reshape((39999,3072))
test_imgs=np.asarray(test_imgs).reshape((10001,3072))

def next_batch(imgs , labs , batch_size):
    indices=random.sample(range(len(imgs)) , batch_size)
    batch_xs=imgs[indices]
    batch_ys=labs[indices]
    return batch_xs , batch_ys

# preprocessing 모듈에서 train_imgs , train_labs , test_imgs , test_labs 를 불러 온다.
# fully connected 모델을 위해 32x32x3 크기의 이미지를 3072 으로 reshape 한다.
# next_batch 함수는 batch_size 수 만큼 random 으로 추출하여 train batch 를 생성하는 함수

# Set Placeholder And Variables

In [3]:
n_classes = 10
learning_rate=0.001
max_iter=100000
check_point=1000

height = 32
width = 32
color_ch=3

x_ =tf.placeholder(tf.float32, [ None , height * width * color_ch ])
y_ =tf.placeholder( tf.int32 , [ None , n_classes ])

# Placeholder 는 차후에 입력할 값
# Variables 는 weight 로 학습시 변하는 값

# Network Modeling

In [4]:
out_dim1 =3072
w1=tf.get_variable("w1" , [height * width * color_ch, out_dim1] , initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.constant(0.1) ,out_dim1)

layer1=tf.matmul(x_,w1) + b1
layer1=tf.nn.relu(layer1)


out_dim2 =1024
w2=tf.get_variable("w2" , [out_dim1, out_dim2] , initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.constant(0.1) ,out_dim2)

layer2=tf.matmul(layer1,w2) + b2
layer2=tf.nn.relu(layer2)

out_dim3 =10
w3=tf.get_variable("w3" , [out_dim2, out_dim3] , initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.constant(0.1) ,out_dim3)

layer3=tf.matmul(layer2,w3) + b3
layer3=tf.nn.relu(layer3)

# 3개의 neural network 로 모델을 구성
# input dimension 과 weight 의 dimension 이 일치해야함
# weight 의 initializer 는 xavier initializer 를 사용
# activation 함수는 ReLU를 사용

# Define Functions

In [5]:
pred=tf.nn.softmax(layer3)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred , labels=y_) , name='cost')
train=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost,name='train')
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32), name='accuracy')

# softmax : 결과값을 각 class 에 해당하는 확률값을로 리턴
# softmax_cross_entropy_with_logits : 실제 정답인 onehot vector 와 예측값 pred 를 차이를 cross_entropy 로 계산
# tf.train.GradientDescentOptimizer : cost 가 최소와 되도록 weight를 조정하는 함수
# accuracy : 실제 값과 예측값의 일치률

# Training

In [6]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

batch_iteration = 100    
training_epochs = 2000

train_cost_list = []
test_cost_list = []
train_acc_list = []
test_acc_list = []

# Tensorflow 학습을 위한 session 정의
# saver : 학습한 weight 와 graph 를 저장

# sess.run 을 통해 함수를 실행 , feed_dict 는 placeholder 에 들어갈 값


for epoch in range(training_epochs):
    
    train_avg_cost = 0.
    train_avg_acc = 0.
    test_avg_cost = 0.
    test_avg_acc = 0.
    
    for batch in range(batch_iteration) :
        batch_xs , batch_ys = next_batch(train_imgs , train_labs , 60)
        
        sess.run(train, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_cost += sess.run(cost, feed_dict={x_:batch_xs , y_ : batch_ys})
        train_avg_acc += sess.run(accuracy, feed_dict={x_:batch_xs , y_ : batch_ys})
        
    train_avg_cost = train_avg_cost / batch_iteration
    train_avg_acc = train_avg_acc / batch_iteration
    
    test_avg_cost = sess.run(cost, feed_dict={x_: test_imgs, y_: test_labs})
    test_avg_acc = sess.run(accuracy, feed_dict={x_: test_imgs, y_: test_labs})
    
    
    print "##################################"
    print ("Epoch: %03d/%03d train cost: %.4f" % (epoch, training_epochs, train_avg_cost))
    print ("Epoch: %03d/%03d train acc: %.4f" % (epoch, training_epochs, train_avg_acc))
    print ("Epoch: %03d/%03d test cost: %.4f" % (epoch, training_epochs, test_avg_cost))
    print ("Epoch: %03d/%03d test acc: %.4f" % (epoch, training_epochs, test_avg_acc))    
    
    train_cost_list.append(train_avg_cost)
    test_cost_list.append(test_avg_cost)
    train_acc_list.append(train_avg_acc)
    test_acc_list.append(test_avg_acc)
   
    
    
saver.save(sess, "model/cifar_fully_connected.ckpt")

with open('cost_acc/cifar_fully_connected_train_cost', 'wb') as fp:
    pickle.dump(train_cost_list, fp)
with open('cost_acc/cifar_fully_connected_test_cost', 'wb') as fp:
    pickle.dump(test_cost_list, fp)
with open('cost_acc/cifar_fully_connected_train_acc', 'wb') as fp:
    pickle.dump(train_acc_list, fp)
with open('cost_acc/cifar_fully_connected_test_acc', 'wb') as fp:
    pickle.dump(test_acc_list, fp)

##################################
Epoch: 000/2000 train cost: 2.3041
Epoch: 000/2000 train acc: 0.1083
Epoch: 000/2000 test cost: 2.3035
Epoch: 000/2000 test acc: 0.0921
##################################
Epoch: 001/2000 train cost: 2.3027
Epoch: 001/2000 train acc: 0.1063
Epoch: 001/2000 test cost: 2.3011
Epoch: 001/2000 test acc: 0.1091
##################################
Epoch: 002/2000 train cost: 2.3010
Epoch: 002/2000 train acc: 0.1003
Epoch: 002/2000 test cost: 2.2987
Epoch: 002/2000 test acc: 0.1028
##################################
Epoch: 003/2000 train cost: 2.2975
Epoch: 003/2000 train acc: 0.1033
Epoch: 003/2000 test cost: 2.2962
Epoch: 003/2000 test acc: 0.1020
##################################
Epoch: 004/2000 train cost: 2.2953
Epoch: 004/2000 train acc: 0.1020
Epoch: 004/2000 test cost: 2.2939
Epoch: 004/2000 test acc: 0.1020
##################################
Epoch: 005/2000 train cost: 2.2940
Epoch: 005/2000 train acc: 0.0995
Epoch: 005/2000 test cost: 2.2919
Epoch: 

##################################
Epoch: 048/2000 train cost: 2.2296
Epoch: 048/2000 train acc: 0.2378
Epoch: 048/2000 test cost: 2.2286
Epoch: 048/2000 test acc: 0.2364
##################################
Epoch: 049/2000 train cost: 2.2295
Epoch: 049/2000 train acc: 0.2398
Epoch: 049/2000 test cost: 2.2286
Epoch: 049/2000 test acc: 0.2349
##################################
Epoch: 050/2000 train cost: 2.2208
Epoch: 050/2000 train acc: 0.2427
Epoch: 050/2000 test cost: 2.2276
Epoch: 050/2000 test acc: 0.2375
##################################
Epoch: 051/2000 train cost: 2.2317
Epoch: 051/2000 train acc: 0.2348
Epoch: 051/2000 test cost: 2.2272
Epoch: 051/2000 test acc: 0.2375
##################################
Epoch: 052/2000 train cost: 2.2252
Epoch: 052/2000 train acc: 0.2430
Epoch: 052/2000 test cost: 2.2267
Epoch: 052/2000 test acc: 0.2366
##################################
Epoch: 053/2000 train cost: 2.2251
Epoch: 053/2000 train acc: 0.2433
Epoch: 053/2000 test cost: 2.2263
Epoch: 

##################################
Epoch: 096/2000 train cost: 2.2120
Epoch: 096/2000 train acc: 0.2507
Epoch: 096/2000 test cost: 2.2127
Epoch: 096/2000 test acc: 0.2508
##################################
Epoch: 097/2000 train cost: 2.2102
Epoch: 097/2000 train acc: 0.2552
Epoch: 097/2000 test cost: 2.2125
Epoch: 097/2000 test acc: 0.2511
##################################
Epoch: 098/2000 train cost: 2.2169
Epoch: 098/2000 train acc: 0.2443
Epoch: 098/2000 test cost: 2.2126
Epoch: 098/2000 test acc: 0.2502
##################################
Epoch: 099/2000 train cost: 2.2122
Epoch: 099/2000 train acc: 0.2570
Epoch: 099/2000 test cost: 2.2121
Epoch: 099/2000 test acc: 0.2518
##################################
Epoch: 100/2000 train cost: 2.2144
Epoch: 100/2000 train acc: 0.2507
Epoch: 100/2000 test cost: 2.2125
Epoch: 100/2000 test acc: 0.2514
##################################
Epoch: 101/2000 train cost: 2.2088
Epoch: 101/2000 train acc: 0.2528
Epoch: 101/2000 test cost: 2.2122
Epoch: 

##################################
Epoch: 144/2000 train cost: 2.2008
Epoch: 144/2000 train acc: 0.2617
Epoch: 144/2000 test cost: 2.2053
Epoch: 144/2000 test acc: 0.2562
##################################
Epoch: 145/2000 train cost: 2.2033
Epoch: 145/2000 train acc: 0.2592
Epoch: 145/2000 test cost: 2.2052
Epoch: 145/2000 test acc: 0.2559
##################################
Epoch: 146/2000 train cost: 2.2066
Epoch: 146/2000 train acc: 0.2545
Epoch: 146/2000 test cost: 2.2051
Epoch: 146/2000 test acc: 0.2556
##################################
Epoch: 147/2000 train cost: 2.2000
Epoch: 147/2000 train acc: 0.2638
Epoch: 147/2000 test cost: 2.2050
Epoch: 147/2000 test acc: 0.2556
##################################
Epoch: 148/2000 train cost: 2.2096
Epoch: 148/2000 train acc: 0.2523
Epoch: 148/2000 test cost: 2.2050
Epoch: 148/2000 test acc: 0.2569
##################################
Epoch: 149/2000 train cost: 2.2094
Epoch: 149/2000 train acc: 0.2538
Epoch: 149/2000 test cost: 2.2047
Epoch: 

##################################
Epoch: 192/2000 train cost: 2.2038
Epoch: 192/2000 train acc: 0.2588
Epoch: 192/2000 test cost: 2.2006
Epoch: 192/2000 test acc: 0.2577
##################################
Epoch: 193/2000 train cost: 2.1981
Epoch: 193/2000 train acc: 0.2623
Epoch: 193/2000 test cost: 2.2003
Epoch: 193/2000 test acc: 0.2593
##################################
Epoch: 194/2000 train cost: 2.1980
Epoch: 194/2000 train acc: 0.2637
Epoch: 194/2000 test cost: 2.2004
Epoch: 194/2000 test acc: 0.2591
##################################
Epoch: 195/2000 train cost: 2.2008
Epoch: 195/2000 train acc: 0.2622
Epoch: 195/2000 test cost: 2.2003
Epoch: 195/2000 test acc: 0.2590
##################################
Epoch: 196/2000 train cost: 2.2022
Epoch: 196/2000 train acc: 0.2567
Epoch: 196/2000 test cost: 2.2002
Epoch: 196/2000 test acc: 0.2596
##################################
Epoch: 197/2000 train cost: 2.1940
Epoch: 197/2000 train acc: 0.2672
Epoch: 197/2000 test cost: 2.2004
Epoch: 

...

##################################
Epoch: 1807/2000 train cost: 2.1542
Epoch: 1807/2000 train acc: 0.3038
Epoch: 1807/2000 test cost: 2.1627
Epoch: 1807/2000 test acc: 0.2916
##################################
Epoch: 1808/2000 train cost: 2.1398
Epoch: 1808/2000 train acc: 0.3207
Epoch: 1808/2000 test cost: 2.1629
Epoch: 1808/2000 test acc: 0.2904
##################################
Epoch: 1809/2000 train cost: 2.1475
Epoch: 1809/2000 train acc: 0.3187
Epoch: 1809/2000 test cost: 2.1629
Epoch: 1809/2000 test acc: 0.2920
##################################
Epoch: 1810/2000 train cost: 2.1467
Epoch: 1810/2000 train acc: 0.3150
Epoch: 1810/2000 test cost: 2.1635
Epoch: 1810/2000 test acc: 0.2899
##################################
Epoch: 1811/2000 train cost: 2.1439
Epoch: 1811/2000 train acc: 0.3185
Epoch: 1811/2000 test cost: 2.1628
Epoch: 1811/2000 test acc: 0.2900
##################################
Epoch: 1812/2000 train cost: 2.1545
Epoch: 1812/2000 train acc: 0.3100
Epoch: 1812/2000 te

##################################
Epoch: 1854/2000 train cost: 2.1412
Epoch: 1854/2000 train acc: 0.3208
Epoch: 1854/2000 test cost: 2.1625
Epoch: 1854/2000 test acc: 0.2914
##################################
Epoch: 1855/2000 train cost: 2.1517
Epoch: 1855/2000 train acc: 0.3095
Epoch: 1855/2000 test cost: 2.1628
Epoch: 1855/2000 test acc: 0.2905
##################################
Epoch: 1856/2000 train cost: 2.1410
Epoch: 1856/2000 train acc: 0.3252
Epoch: 1856/2000 test cost: 2.1623
Epoch: 1856/2000 test acc: 0.2899
##################################
Epoch: 1857/2000 train cost: 2.1507
Epoch: 1857/2000 train acc: 0.3130
Epoch: 1857/2000 test cost: 2.1621
Epoch: 1857/2000 test acc: 0.2919
##################################
Epoch: 1858/2000 train cost: 2.1368
Epoch: 1858/2000 train acc: 0.3302
Epoch: 1858/2000 test cost: 2.1624
Epoch: 1858/2000 test acc: 0.2900
##################################
Epoch: 1859/2000 train cost: 2.1335
Epoch: 1859/2000 train acc: 0.3338
Epoch: 1859/2000 te

##################################
Epoch: 1901/2000 train cost: 2.1445
Epoch: 1901/2000 train acc: 0.3193
Epoch: 1901/2000 test cost: 2.1620
Epoch: 1901/2000 test acc: 0.2908
##################################
Epoch: 1902/2000 train cost: 2.1430
Epoch: 1902/2000 train acc: 0.3203
Epoch: 1902/2000 test cost: 2.1615
Epoch: 1902/2000 test acc: 0.2933
##################################
Epoch: 1903/2000 train cost: 2.1421
Epoch: 1903/2000 train acc: 0.3207
Epoch: 1903/2000 test cost: 2.1619
Epoch: 1903/2000 test acc: 0.2917
##################################
Epoch: 1904/2000 train cost: 2.1424
Epoch: 1904/2000 train acc: 0.3200
Epoch: 1904/2000 test cost: 2.1619
Epoch: 1904/2000 test acc: 0.2911
##################################
Epoch: 1905/2000 train cost: 2.1442
Epoch: 1905/2000 train acc: 0.3188
Epoch: 1905/2000 test cost: 2.1620
Epoch: 1905/2000 test acc: 0.2918
##################################
Epoch: 1906/2000 train cost: 2.1418
Epoch: 1906/2000 train acc: 0.3217
Epoch: 1906/2000 te

##################################
Epoch: 1948/2000 train cost: 2.1403
Epoch: 1948/2000 train acc: 0.3223
Epoch: 1948/2000 test cost: 2.1613
Epoch: 1948/2000 test acc: 0.2919
##################################
Epoch: 1949/2000 train cost: 2.1505
Epoch: 1949/2000 train acc: 0.3132
Epoch: 1949/2000 test cost: 2.1617
Epoch: 1949/2000 test acc: 0.2906
##################################
Epoch: 1950/2000 train cost: 2.1332
Epoch: 1950/2000 train acc: 0.3295
Epoch: 1950/2000 test cost: 2.1613
Epoch: 1950/2000 test acc: 0.2927
##################################
Epoch: 1951/2000 train cost: 2.1509
Epoch: 1951/2000 train acc: 0.3108
Epoch: 1951/2000 test cost: 2.1611
Epoch: 1951/2000 test acc: 0.2916
##################################
Epoch: 1952/2000 train cost: 2.1493
Epoch: 1952/2000 train acc: 0.3115
Epoch: 1952/2000 test cost: 2.1615
Epoch: 1952/2000 test acc: 0.2906
##################################
Epoch: 1953/2000 train cost: 2.1457
Epoch: 1953/2000 train acc: 0.3157
Epoch: 1953/2000 te

##################################
Epoch: 1995/2000 train cost: 2.1442
Epoch: 1995/2000 train acc: 0.3205
Epoch: 1995/2000 test cost: 2.1610
Epoch: 1995/2000 test acc: 0.2914
##################################
Epoch: 1996/2000 train cost: 2.1442
Epoch: 1996/2000 train acc: 0.3202
Epoch: 1996/2000 test cost: 2.1609
Epoch: 1996/2000 test acc: 0.2919
##################################
Epoch: 1997/2000 train cost: 2.1358
Epoch: 1997/2000 train acc: 0.3275
Epoch: 1997/2000 test cost: 2.1610
Epoch: 1997/2000 test acc: 0.2925
##################################
Epoch: 1998/2000 train cost: 2.1424
Epoch: 1998/2000 train acc: 0.3190
Epoch: 1998/2000 test cost: 2.1606
Epoch: 1998/2000 test acc: 0.2917
##################################
Epoch: 1999/2000 train cost: 2.1468
Epoch: 1999/2000 train acc: 0.3147
Epoch: 1999/2000 test cost: 2.1621
Epoch: 1999/2000 test acc: 0.2904
